Tras muchos problemas tratando los datos de medida, he decidido simplificar la vida de una comercializadora y reducir mi data set.

Me voy a quedar solamente con aquellos puntos de los que tengo curva horaria completa para el periodo de dos años (2017-2018), y lo voy a reducir a tres zonas geograficas (Norte, Centro y Sur)

# Importar librerias necesarias

In [ ]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import colors
import numpy as np
import numpy.ma as ma
import seaborn as sns

plt.style.use('ggplot')

# Lectura de los datos

Tengo toda la información de curvas de medida en dos ficheros excel, cada uno de los ficheros tiene varias pestañas por lo que tengo que unir toda la información en un solo data frame.

Lo primero de todo es abrir el primer fichero, ver cuantas pestañas tiene y conocer los nombres de cada una de las pestañas.

Por motivos de confidencialidad estos fichero originales de curvas de consumo horarias por cliente no las puedo subir a github, subiré solo la curva final agregada para 2017 y 2018 por comunidad y por tarifa.

In [ ]:
excel_document_17 = openpyxl.load_workbook('./data/Cups2017.xlsx')

In [ ]:
excel_document_17.get_sheet_names()

### Unificar los datos. 

Accedemos al fichero excel y creamos un data frame de cada una de las pestaña, a continuación unimos todos los data frame en uno solo.

In [ ]:
xlsx_2017 = pd.ExcelFile('./data/Cups2017.xlsx') 

df1 = pd.read_excel(xlsx_2017, 'Page 1') 
df2 = pd.read_excel(xlsx_2017, 'Page 2') 
df3 = pd.read_excel(xlsx_2017, 'Page 3') 
df4 = pd.read_excel(xlsx_2017, 'Page 4')

In [ ]:
pd.read_excel?

In [ ]:
df_2017 = pd.concat([df1, df2, df3, df4])
df_2017.head(5)

Comprobamos que la unión de los ficheros es correcta y que el resultado df_2017 es la suma de los cuatro df anteriores

In [ ]:
df1.count()

In [ ]:
df2.count()

In [ ]:
df3.count()

In [ ]:
df4.count()

In [ ]:
df_2017.count()

## Hacemos el mismo proceso con el excel de 2018

In [ ]:
excel_document_18 = openpyxl.load_workbook('./data/Cups2018.xlsx')
excel_document_18.get_sheet_names()

In [ ]:
xlsx_18 = pd.ExcelFile('./data/Cups2018.xlsx') 
df11 = pd.read_excel(xlsx_18, 'Page 1') 
df22 = pd.read_excel(xlsx_18, 'Page 2') 
df33 = pd.read_excel(xlsx_18, 'Page 3') 
df44 = pd.read_excel(xlsx_18, 'Page 4') 
df55 = pd.read_excel(xlsx_18, 'Page 5')
df_2018 = pd.concat([df11, df22, df33, df44, df55])
df_2018.head(5)

# Unimos tanto el fichero de 2017 como el de 2018 y creamos un único fichero con las medidas de 2017 y 2018

Comprobamos que el fichero final es efectivamente la suma de los dos ficheros originales.

In [ ]:
Medidas = pd.concat([df_2017, df_2018])

In [ ]:
df_2017.shape

In [ ]:
df_2018.shape

In [ ]:
Medidas.shape

### Formato

Damos un formato mas amigable al fichero cambiando los nombres de las columnas y dando formato fecha correcto.


In [ ]:
Medidas = Medidas.rename(columns={'--Cups20--' : 'Cups20',
                                  '--Line--' : 'Line', 
                                  '--Fecha--' : 'Fecha', 
                                  '--Hora--' : 'Hora', 
                                  '--Active--' : 'Active' , 
                                  '--Active With Transport Loss--' : 'Active With Transport Loss', 
                                  '--Active With Transform Loss--' : 'Active With Transform Loss'})

In [ ]:
Medidas.head(5)

In [ ]:
Medidas['Fecha']=pd.to_datetime(Medidas['Fecha'], yearfirst=True,format='%Y-%m-%d')


In [ ]:
Medidas.dtypes

In [ ]:
Medidas.head(5)

# Contratos

A este fichero de medidas le falta información como la provincia por cliente, la tarifa contratada, o la potencia. Todos estos datos los conseguimos del excel de contratos que vamos a unir a este fichero según el CUPS.

In [ ]:
contratos = pd.read_excel('./data/Contratos.xlsx')
contratos.head(5)

In [ ]:
contratos = contratos.rename(columns={'CUPS 20' : 'Cups20'})
contratos.head(5)

In [ ]:
Medidas.shape

Comprobamos que despues del merge tengo las mismas líneas que tenía antes, y que no tengo ningún valor NaN

In [ ]:
Medidas = Medidas.merge(contratos, on='Cups20', how='left')


In [ ]:
Medidas.shape

In [ ]:
Medidas.isnull().any().any()


### Limpieza de datos

Vamos a ver las columnas que tiene el data frame y vamos a quedarnos solo con las necesarias para el analisis.


In [ ]:
Medidas.columns

In [ ]:
Medidas = Medidas[['Cups20', 'Tarifa', 'Provincia','Fecha', 'Hora', 'Active', 'Potencia ']]
Medidas.head(5)

Voy a crear una sola fila de Fecha_Hora y voy a crear ademas una columna con el año, y con el mes. Para esto voy a guardar el fichero y lo voy a leer parseando la fecha para que me la una automaticamente.

In [ ]:
Medidas.to_csv('Medidas.csv')

In [ ]:
Medidas_p = pd.read_csv('./Medidas.csv', parse_dates=[['Fecha', 'Hora']]) 

In [ ]:
Medidas_p.head()

In [ ]:
Medidas_p['Fecha_Hora']=pd.to_datetime(Medidas_p['Fecha_Hora'], yearfirst=True,format='%Y/%m/%d %H')

Medidas_p.head

### Guardamos el fichero como un csv con las medidas de todos nuestros clientes durante dos años.

In [ ]:
# Lo edito para no volver a cargarme el fichero bueno y tener que procesar todo el notebook otra vez que es un infierno.

# Medidas.to_csv('Medidas.csv')

In [ ]:
# Solo para evitar procesar todo lo anterior, si no hace falta comentar lineas

Medidas = pd.read_csv('./Medidas.csv')
Medidas.head(5)

In [ ]:
# Medidas_p=pd.read_csv('./Medidas.csv', parse_dates=[['Fecha', 'Hora']], index_col=None)

# Agrupamos por Fecha, Hora, Provincia, Tarifa

Por motivmos de confidencialidad no puedo mostrar el origen de los datos, que son curvas de consumo horario por CUPS. Sin embargo a partir de ahora trabajermos con curvas de consumo horario agrupadas por estos campos que son suficientes para el desarrollo del modelo.


In [ ]:
Medidas.shape

Como no tengo curva de consumo completa para todos los puntos para todas las horas de los dos años, voy a calcular el número de puntos por hora y la suma de la energía para cada hora, de esta manera si divido la suma de la energía entre el número de puntos, tendría un consumo medio para cada uno de los grupos que voy a tener.

In [ ]:
puntos = Medidas.groupby(['Provincia', 'Fecha', 'Hora'], sort=True).count()

puntos = puntos.reset_index()

puntos.to_csv('puntos.csv')

In [ ]:
puntos.head()

In [ ]:
# !conda update seaborn

In [ ]:
sns.__version__

In [ ]:
sns.catplot('Cups20',data=puntos,kind='count',
            hue='Fecha',col='Provincia',
            palette='Set1');

In [ ]:
plt.plot(puntos['Cups20'])

In [ ]:
Medidas_procesadas=Medidas.groupby(['Provincia', 'Fecha', 'Hora']).agg[{'Cups20': np.sum, 'Active': [np.sum, np.mean]}]

In [ ]:
Medidas_procesadas=Medidas.groupby(['Provincia', 'Fecha', 'Hora'])[['Active']].agg(['sum', 'count'])

In [ ]:
Medidas_procesadas=Medidas_procesadas.reset_index()



In [ ]:
Medidas_procesadas.head()
Medidas_procesadas.to_excel('Medidas_procesadas.xlsx')

In [ ]:
puntos = Medidas.groupby(['Provincia', 'Fecha', 'Hora'], sort=True)
Medida_procesada=(puntos['Active'].agg([ma.count, np.sum]).rename(columns={'count':'NºClientes','sum':'Activa'}) )
Medida_procesada['Tanto1']=Medida_procesada['Activa']/Medida_procesada['NºClientes']

Medida_procesada.describe()

In [ ]:
Medida_procesada=Medida_procesada.reset_index()
Medida_procesada.head(5)

In [ ]:
Medida_procesada.to_excel('Medida_procesada.xlsx')

In [ ]:
Comunidad = pd.read_excel('./data/Comunidades.xlsx')
Comunidad.head(5)

In [ ]:
Medida_procesada = pd.merge(Medida_procesada, Comunidad, on='Provincia')
Medida_procesada.head(5)

In [ ]:
Medida_procesada.index=Medida_procesada['Fecha']
Medida_procesada.head(5)

In [ ]:
Medida_procesada['Fecha']=pd.to_datetime(Medida_procesada['Fecha'], yearfirst=True,format='%Y-%m-%d')

In [ ]:
Medida_procesada.head(5)

In [ ]:
Medida_procesada.cumsum()

In [ ]:
plt.plot(Medida_procesada['Tanto1'])

In [ ]:
Medida_procesada.to_csv('Medida_procesada.csv')

In [ ]:
Medida_procesada.to_excel('Medida_procesada.xlsx')

In [ ]:
# Quitamos la linea del CUPS para quedarnos con el CUPS 20
#df['CUPS 22'] = df['CUPS 22'].str.split('0P', expand=True)
#df['CUPS 20'] = df['CUPS 22']
#df.head(3)